In [1]:
import pandas as pd
import os

In [26]:
line_list_dir = "/user/jcarlberg/research/stell_param_deriv/metal_line_list/"

list1 = "melendez2014_table1_noFe.txt"
list2 = "jkc_thesis_metal_ll.txt"


min_wav = 4850.0
max_wav = 9150.0

In [33]:
line_list1 = pd.read_csv(os.path.join(line_list_dir, list1), delim_whitespace=True,skiprows=25, usecols = [0,1,2,3],names=["Wavelength", "Ion", "EP", "Log_gf"])
line_list1 = line_list1[(line_list1["Wavelength"] < max_wav) & (line_list1["Wavelength"] > min_wav)]
species_list1 = pd.read_csv(os.path.join(line_list_dir, list1), delimiter='#',skiprows=1, usecols = [1],header=None)

line_list2 = pd.read_csv(os.path.join(line_list_dir, list2),delim_whitespace=True, skiprows=1,header=None, usecols=[0,1,2,3],names=["Wavelength", "Ion", "EP", "Log_gf"])
line_list2 = line_list2[(line_list2["Wavelength"] < max_wav) & (line_list2["Wavelength"] > min_wav)]


In [34]:
frames=[line_list1, line_list2]
master_line_list = pd.concat(frames).drop_duplicates(subset="Wavelength").sort_values("Wavelength")

ion_list = ['I',"II",'III','IV','V','VI']
species_list = ["H","He", "Li",'Be','B','C','N','O','F','Ne',
               'Na','Mg','Al','Si','P','S','Cl','Ar','K','Ca',
               'Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu',
               'Zn','Ga','Ge','As','Se','Br','Kr','Rb','Sr','Y',
               'Zr','Nb','Mo','Tc','Ru','Rh','Pd','Ag','Cd','In',
               'Sn','Sb','Te','I','Xe','Cs','Ba','La','Ce','Pr',
               'Nd','Pm','Sm','Eu', 'Gd','Tb','Dy','Ho','Er','Tm',
               'Yb']
len(species_list)

70

In [35]:
#species = [species_list[int(str(ion).split('.')[0])-1]+' '+ ion_list[int(str(ion).split('.')[1])-1] for ion in list(master_line_list.sort_values("Ion")['Ion'])]
master_line_list['Species'] = [species_list[int(str(ion).split('.')[0])-1]+' '+ ion_list[int(str(ion).split('.')[1])] for ion in list(master_line_list['Ion'])]


In [36]:
master_line_list.sort_values('Ion').to_csv("master_line_list.csv",index=False)

In [37]:
master_line_list.sort_values('Wavelength')

,Wavelength,Ion,EP,Log_gf,Species
166,4854.867,39.1,0.9923,-0.380,Y II
88,4865.611,22.1,1.1160,-2.810,Ti II
89,4874.014,22.1,3.0950,-0.900,Ti II
95,4875.486,23.0,0.0400,-0.810,V I
167,4883.685,39.1,1.0841,0.070,Y II
168,4900.110,39.1,1.0326,-0.090,Y II
90,4911.193,22.1,3.1230,-0.537,Ti II
69,4913.616,22.0,1.8731,0.161,Ti I
105,4936.335,24.0,3.1128,-0.250,Cr I
139,4953.208,28.0,3.7397,-0.660,Ni I


In [21]:
ion_list

['I', 'II', 'III', 'IV', 'V', 'VI']